In [ ]:
#%%
import sys
sys.path.append("../")

import numpy as np
from tensorflow import keras
from oodeel.utils.load_utils import get_model
from oodeel.methods.mls import MLS
from oodeel.methods.dknn import DKNN
import tensorflow as tf
import matplotlib.pyplot as plt


## Data and model loading
ID data: MNIST, OOD data: Fashion MNIST

In [ ]:

num_classes = 10
input_shape = (28, 28, 1)

#### MNIST

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#### Fashion MNIST

# Load the data and split it between train and test sets
_ , (x_ood, y_ood) = keras.datasets.fashion_mnist.load_data()

# Scale images to the [0, 1] range
x_ood = x_ood.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_ood = np.expand_dims(x_ood, -1)


Will also monitor scores on classifier error (because why not)

In [ ]:

model = get_model("../models/save/mnist_model")
y_pred = model.predict(x_test)
maxs = np.max(y_pred, axis=1)

for i, y in enumerate(y_pred):
    for j, yy in enumerate( y):
        if yy == maxs[i]:
            y_pred[i,j] = 1
        else:
            y_pred[i,j] =  0

errors = np.sum(np.abs(y_test - y_pred), axis=1)/2 

## Maximum Logit Score
(A good classifier is all you need ?)

Example of method that does not need ID data to compute statistics to compare against

In [ ]:

oodmodel = MLS(model)
scores_id = oodmodel.score(x_test)
scores_ood = oodmodel.score(x_ood)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.hist((scores_id[np.where(errors == 1)], scores_id[np.where(errors == 0)]), bins=10, color=("blue", "orange"), label=("error", "good"), density=True)
plt.legend()

plt.subplot(1,2,2)
plt.hist((scores_ood, scores_id), bins=10, color=("blue", "orange"), label=("ood", "id"), density=True)

plt.legend()
plt.show()

## Deep KNN
Example of method that needs ID data to compute statistics to compare against

In [ ]:
%load_ext autoreload
%autoreload 2

oodmodel = DKNN(model)
oodmodel.fit(x_train[:20000])

In [ ]:
scores_id = oodmodel.score(x_test[:1000], 30)
scores_ood = oodmodel.score(x_ood[:1000], 30)

In [ ]:
plt.figure(figsize=(8,5))

plt.hist((scores_ood, scores_id), bins=10, color=("blue", "orange"), label=("ood", "idd"), density=True)
plt.legend()
plt.show()